# 🎉 QAG Sentiment Analysis: Understanding Sentiments in French Political Interventions 🇫🇷

Welcome to the QAG Sentiment Analysis notebook! This notebook aims to analyze the sentiment of interventions during the "Questions Au Gouvernement" (QAG) sessions in French politics. By leveraging Natural Language Processing (NLP) techniques and state-of-the-art pre-trained models, we'll infer the sentiment of each intervention to better understand the overall tone of the discussions. 🗣️

## 🔍 Overview

We will be using the Hugging Face Transformers library and the "cmarkea/distilcamembert-base-sentiment" model to perform sentiment analysis on the QAG dataset. This pre-trained model can efficiently handle multilingual text inputs and provide accurate sentiment predictions. The sentiment scores will be appended to the original dataset, offering a comprehensive view of the sentiment landscape in QAG sessions. 📊

## 📝 Steps

1. Install the Transformers library. 📦
2. Import the necessary libraries and load the QAG dataset. 📚
3. Create a sentiment analysis pipeline using the pre-trained multilingual sentiment model. 🧪
4. Test the sentiment analysis pipeline with sample sentences. 📖
5. Define a custom function (`sentiment_to_df`) to tokenize, truncate, and infer the sentiment of each intervention in the dataset. 🛠️
6. Apply the custom function to the entire dataset, creating new columns for sentiment scores. 📈
7. Save the updated dataset with sentiment scores as a new CSV file. 💾

## 📌 Results

Once the sentiment analysis is complete, the updated dataset will contain the sentiment scores for each intervention. This information can be utilized to visualize trends, perform further analysis, and gain insights into the dynamics of the QAG sessions. 📉

````
   session_date  ... sentiment_score
0    2021-12-15  ...        0.987380
1    2021-12-15  ...        0.999869
2    2021-12-15  ...        0.999779
3    2021-12-15  ...        0.998926
4    2021-12-15  ...        0.996955

[5 rows x 7 columns]
````


This notebook provides an effective and efficient way to analyze the sentiment of political interventions during QAG sessions, offering valuable insights into the tone and mood of these important discussions. 💡🌟



# Install transformers

In [1]:
!pip install transformers
!pip install sentencepiece 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# import libs and load dataset

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
tqdm.pandas()
from transformers import pipeline

In [3]:
df = pd.read_csv("/content/drive/MyDrive/AI/QAG/raw_qag/complete_qag_15.csv", index_col=0)

In [4]:
df.head()

,legislature_number,official_date,qag_number,intervention_number,speaker_name,intervention_sentences
0,15,11/04/2018,806,0,M. le président.,"La parole est à Mme Sophie Auconie, pour le gr..."
1,15,11/04/2018,806,1,Mme Sophie Auconie.,Madame la ministre des solidarités et de la sa...
2,15,11/04/2018,806,2,Mme Danièle Obono.,Très bien !
3,15,11/04/2018,806,3,Mme Sophie Auconie.,Il est temps de nous préoccuper du mal-être de...
4,15,11/04/2018,806,4,M. Éric Coquerel et Mme Laurence Dumont .,Très bien !


In [5]:
model_path = "cmarkea/distilcamembert-base-sentiment"

sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
sentiment_task("Covid cases are increasing fast!")
sentiment_task("Ce fromage est dégeulasse")
sentiment_task("J'adore les fleurs")

[{'label': '5 stars', 'score': 0.48123905062675476}]

In [6]:
import torch

In [7]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path, device=0)

def sentiment_to_df(sentences):
    tokens = sentiment_pipeline.tokenizer.encode(sentences, truncation=True, max_length=512)
    input_ids = torch.tensor(tokens).unsqueeze(0).to(sentiment_pipeline.device)  # Move input tensor to GPU
    outputs = sentiment_pipeline.model(input_ids)
    result = outputs.logits.detach().cpu().numpy()  # Move output tensor back to CPU
    stars = np.argmax(result, axis=1)[0]
    score = np.max(result, axis=1)[0]

    return [stars, score]

df[['sentiment_stars', 'sentiment_score']] = df.progress_apply(lambda row: sentiment_to_df(row['intervention_sentences']), axis=1, result_type="expand")

  0%|          | 0/57680 [00:00<?, ?it/s]

In [8]:
df.head()

,legislature_number,official_date,qag_number,intervention_number,speaker_name,intervention_sentences,sentiment_stars,sentiment_score
0,15,11/04/2018,806,0,M. le président.,"La parole est à Mme Sophie Auconie, pour le gr...",4.0,0.889632
1,15,11/04/2018,806,1,Mme Sophie Auconie.,Madame la ministre des solidarités et de la sa...,4.0,2.656831
2,15,11/04/2018,806,2,Mme Danièle Obono.,Très bien !,4.0,2.782947
3,15,11/04/2018,806,3,Mme Sophie Auconie.,Il est temps de nous préoccuper du mal-être de...,4.0,0.780742
4,15,11/04/2018,806,4,M. Éric Coquerel et Mme Laurence Dumont .,Très bien !,4.0,2.782947


In [9]:
df.to_csv("/content/drive/MyDrive/AI/QAG/sentiment/camenbert_sentiment_qag_15.csv")

# Check that Good sentiment is 4-5 stars

In [10]:
set(df[df.sentiment_stars == 4].intervention_sentences)

{"Monsieur\xa0le ministre d'État, ministre de l'intérieur, vous qui avez été l'un des premiers maires à rejoindre le mouvement En marche, vous n'avez pu que vous réjouir de l'élection, dimanche 2\xa0décembre, du premier maire sous les couleurs de la République en marche, à Saint-Sulpice, dans le Tarn. Nous pouvons sincèrement nous réjouir de cette élection, qui est le résultat de plusieurs facteurs. En premier lieu, c'est le choix d'un projet communal porté par une équipe composée d'hommes et de femmes engagés auprès d'En marche et d'autres qui ne le sont pas. Cette diversité de parcours et de sensibilités politiques nous rappelle que c'est cela, l'ADN de la République en marche. En second lieu, cette élection est bel et bien le résultat du respect de la parole que nous avons donnée aux Français : nous avons mis en place les mesures du programme présidentiel sur lequel nous avons été élus. Le bilan des actions conduites durant les six premiers mois du mandat montre que nous avons tenu 

In [11]:
set(df[df.sentiment_stars == 0].intervention_sentences)

{"…car ce serait une erreur stratégique de continuer à mettre tous nos œufs dans le même panier. Dans le cadre du plan France\xa02030, le Président de la République a annoncé 1\xa0milliard d'euros d'investissements dans la recherche et développement en faveur du nucléaire. Cessez donc de dire que nous n'investissons pas dans ce domaine.Mon problème, en tant que ministre de l'énergie, monsieur le député, c'est que 70\xa0% de notre parc électrique est nucléaire et qu'il n'a jamais aussi peu produit.",
 "Les mesures annoncées hier représentent 9\xa0milliards d'euros, et vous ne rétablissez même pas l'ISF pour les financer, pas plus que vous n'abandonnez la flat tax sur le capital. On vous voit revenir avec vos obsessions de coupes budgétaires\xa0: autrement dit, vous continuerez à détruire les services publics que demandent massivement les gilets jaunes\xa0! Vous n'avez pas compris ce qui vient du pays. C'est une demande de justice fiscale, pour que, enfin, ceux qui gagnent plus paient à 